In [ ]:
pip install sagemaker -U

In [2]:
import os
import boto3
import re
import sagemaker
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

role = sagemaker.get_execution_role()
region = sagemaker.Session().boto_region_name
session = Session()
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sklearn import preprocessing
bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-xgboost-dist-algo"

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## 2. Data Preparation and Visualization

In [5]:
s3 = boto3.client("s3")
s3.download_file(
    f"sagemaker-example-files-prod-{region}",
    "datasets/tabular/synthetic/churn.txt",
    "churn.txt",
)

[01/20/25 17:48:08] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=340329;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=443254;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

In [10]:
import pandas as pd
churn = pd.read_csv("./churn.txt")
pd.set_option("display.max_columns", 500)
churn.head(5)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,PA,163,806,403-2562,no,yes,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,True.
1,SC,15,836,158-8416,yes,no,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,False.
2,MO,131,777,896-6253,no,yes,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,False.
3,WY,75,878,817-5729,yes,yes,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,False.
4,WY,146,878,450-4942,yes,no,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,True.


In [11]:
churn = pd.concat([churn]*100, ignore_index=True) # to increase the size of the table for the purpose of increasing the training data size

In [12]:
churn

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,PA,163,806,403-2562,no,yes,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,True.
1,SC,15,836,158-8416,yes,no,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,False.
2,MO,131,777,896-6253,no,yes,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,False.
3,WY,75,878,817-5729,yes,yes,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,False.
4,WY,146,878,450-4942,yes,no,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,True.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,NH,4,787,151-3162,yes,yes,800,10.862632,5,7.250969,6.936164,1,8.026482,4.921314,350,6.748489,4.872570,8,2.122530,9,False.
499996,SD,140,836,351-5993,no,no,0,1.581127,8,3.758307,7.377591,7,1.328827,0.939932,300,4.522661,6.938571,2,4.600473,4,False.
499997,SC,32,836,370-3127,no,yes,700,0.163836,5,4.243980,5.841852,3,2.340554,0.939469,450,5.157898,4.388328,7,1.060340,6,False.
499998,MA,142,776,604-2108,yes,yes,600,2.034454,5,3.014859,4.140554,3,3.470372,6.076043,150,4.362780,7.173376,3,4.871900,7,True.


In [13]:
churn = churn.drop("Phone", axis=1)
churn["Area Code"] = churn["Area Code"].astype(object)

In [14]:
churn["target"] = churn["Churn?"].map({"True.": 1, "False.": 0})
churn.drop(["Churn?"], axis=1, inplace=True)

In [15]:
churn = churn[["target"] + churn.columns.tolist()[:-1]]

In [16]:
# One Hot Encode Cat Variables
churn=pd.get_dummies(churn, dtype=int)
churn

,target,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area Code_657,Area Code_658,Area Code_659,Area Code_676,Area Code_677,Area Code_678,Area Code_686,Area Code_707,Area Code_716,Area Code_727,Area Code_736,Area Code_737,Area Code_758,Area Code_766,Area Code_776,Area Code_777,Area Code_778,Area Code_786,Area Code_787,Area Code_788,Area Code_797,Area Code_798,Area Code_806,Area Code_827,Area Code_836,Area Code_847,Area Code_848,Area Code_858,Area Code_866,Area Code_868,Area Code_876,Area Code_877,Area Code_878,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,1,163,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,0,15,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
2,0,131,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,0,75,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1
4,1,146,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,4,800,10.862632,5,7.250969,6.936164,1,8.026482,4.921314,350,6.748489,4.872570,8,2.122530,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
499996,0,140,0,1.581127,8,3.758307,7.377591,7,1.328827,0.939932,300,4.522661,6.938571,2,4.600473,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0
499997,0,32,700,0.163836,5,4.243980,5.841852,3,2.340554,0.939469,450,5.157898,4.388328,7,1.060340,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
499998,1,142,600,2.034454,5,3.014859,4.140554,3,3.470372,6.076043,150,4.362780,7.173376,3,4.871900,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0

In [17]:
from sklearn.model_selection import train_test_split

train, val_n_test = train_test_split(
    churn, test_size=0.3, random_state=42, stratify=churn["target"]
)

val, test = train_test_split(
    val_n_test, test_size=0.3, random_state=42, stratify=val_n_test["target"]
)

In [19]:
train.to_parquet('train.parquet')

val.to_parquet('validation.parquet')
test.to_parquet('test.parquet')


In [ ]:
import pandas as pd
import pyarrow.parquet as pq

# Method 1: Using pandas
def check_datatypes_pandas(parquet_file):
    # Read the Parquet file
    df = pd.read_parquet(parquet_file)

    # Print the data types of each column
    print("Data types using pandas:")
    print(df.dtypes)

    # If you want more detailed information
    print("\nDetailed info:")
    df.info()

# Method 2: Using pyarrow
def check_datatypes_pyarrow(parquet_file):
    # Read the Parquet file schema
    table = pq.read_table(parquet_file)
    schema = table.schema

    print("Data types using pyarrow:")
    for field in schema:
        print(f"{field.name}: {field.type}")

# Usage
parquet_file = 'train.parquet'

print("Using pandas:")
check_datatypes_pandas(parquet_file)

print("\nUsing pyarrow:")
check_datatypes_pyarrow(parquet_file)

In [25]:
train.to_csv("train.csv", header=False, index=False)
val.to_csv("validation.csv", header=False, index=False)
test.to_csv("test.csv", header=False, index=False)

For demonstartion purpose on including multiple files under the training channel, we simply duplicate the training data multiple times as shown below.

For the purprose of the training logic implemented that uses `xgboost.rabit()` for distributed training,it is recommended to split training sets into multiple chunks so they can be distributed accross training nodes. However, for single node training, this is not required.

In [21]:
from tqdm import tqdm

for i in tqdm(range(500)):
    boto3.Session().resource("s3").Bucket(bucket).Object(
        os.path.join(prefix, f"train_xgb_parquet/data_{i}.parquet")
    ).upload_file("train.parquet")

100%|██████████| 500/500 [03:54<00:00,  2.13it/s]


In [22]:
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation_xgb_parquet/data.parquet")
).upload_file("validation.parquet")

In [23]:
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "test_xgb_parquet/data.parquet")
).upload_file("test.parquet")

In [24]:
training_dataset_s3_path = f"s3://{bucket}/{prefix}/train_xgb_parquet/"
validation_dataset_s3_path = f"s3://{bucket}/{prefix}/validation_xgb_parquet/"

output_prefix = "xgboost-perf-training"
s3_output_location = f"s3://{bucket}/{output_prefix}/output_xgb"
training_dataset_s3_path, validation_dataset_s3_path

('s3://sagemaker-us-east-1-715253196401/sagemaker/DEMO-xgboost-dist-algo/train_xgb_parquet/',
 's3://sagemaker-us-east-1-715253196401/sagemaker/DEMO-xgboost-dist-algo/validation_xgb_parquet/')

## Create an XGBoost training script

SageMaker can now run an XGBoost script using the XGBoost estimator. When run on SageMaker, a number of helpful environment variables are available to access properties of the training environment, such as:

- `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to. Any artifacts saved in this folder are uploaded to S3 for model hosting after the training job completes.
- `SM_OUTPUT_DIR`: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.

When two input channels, `train` and `validation`, are used in the call to the XGBoost estimator's `fit()` method, the following environment variables are set, following the format `SM_CHANNEL_[channel_name]`:

- `SM_CHANNEL_TRAIN`: A string representing the path to the directory containing data in the 'train' channel.
- `SM_CHANNEL_VALIDATION`: Same as above, but for the 'validation' channel.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to the `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance. For example, the script that we run in this notebook is provided as the accompanying file (`abalone.py`) and also shown below:

In [37]:
!pygmentize -g code/abalone.py

#  Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License").
#  You may not use this file except in compliance with the License.
#  A copy of the License is located at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  or in the "license" file accompanying this file. This file is distributed
#  on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
#  express or implied. See the License for the specific language governing
#  permissions and limitations under the License.
from __future__ import print_function

import argparse
import json
import logging
import os
import pickle as pkl

import pandas as pd
import xgboost as xgb
from sagemaker_containers import entry_point
from sagemaker_xgboost_container import distributed
from sagemaker_xgboost_container.data_utils import get_dmatrix
from sagemaker_xgboost_container.algorithm_mode import hyperparameter_validation as hpv
from sagemak

Because the container imports your training script, always put your training code in a main guard `(if __name__=='__main__':)` so that the container does not inadvertently run your training code at the wrong point in execution.

For more information about training environment variables, please visit the [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit).

In [4]:
instance_type = "ml.m5.xlarge"
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "algo-dist-xgb")
content_type ="csv" #dataset extension
hyperparams = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
    "num_round": "50",
    "verbosity": "3",
    "eval_metric": "auc",
    "content_type": content_type,
    sagemaker_program	"train.py"

}

---
## Train the XGBoost model

After setting training parameters, we kick off training, and poll for status until training is complete.

To run our training script on SageMaker, we construct a sagemaker.xgboost.estimator.XGBoost estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script that SageMaker runs for training and prediction.
* __role__: Role ARN
* __train_instance_type__ *(optional)*: The type of SageMaker instances for training.
* __sagemaker_session__ *(optional)*: The session used to train on SageMaker.
* __hyperparameters__ *(optional)*: A dictionary passed to the train function as hyperparameters.

SageMaker Training Directory Setup for Script Mode:

- Create a root project directory.
- Place main training script (e.g., train.py) in root.
- Add other Python modules/scripts to root or subdirectories.
- Include requirements.txt for dependencies. Sagemaker automatically installs all libs listed in this text file

Example structure:
```
project/
    ├── train.py
    ├── requirements.txt
    ├── utils.py
```

SageMaker estimator setup:
```
estimator = Estimator(
    entry_point='train.py',
    source_dir='path/to/project',
    ...
)
```
Key points:

    Include all necessary code files.
    List dependencies in requirements.txt.
    SageMaker packages entire directory content.    

---

In [64]:
# Open Source distributed script mode
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

session = Session()
script_path = "train.py"
instance_count = 1 # for distributed training, set instance greater than 1
instance_type = "ml.g5.2xlarge"
# Your MLflow tracking ARN
tracking_server_arn = "arn:aws:sagemaker:us-east-1:1234567890:mlflow-tracking-server/test"

xgb_script_mode_estimator = XGBoost(
    source_dir="s3://sagemaker-us-east-1-715253196401/sagemaker-xgboost-2025-01-22-18-58-16-411/source/sourcedir.tar.gz", # parent folder of training logic and dependencies
    entry_point=script_path, # training logic path
    framework_version="1.7-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    volume_size=50,
    instance_count=instance_count, # for distributed training, set instance greater than 1
    instance_type=instance_type,
    output_path=output_path,
        subnets=['subnet-01e40580335386c69'],
    security_group_ids=['sg-04544eabba5214ad3'],
      # environment={"MLFLOW_TRACKING_ARN": tracking_server_arn},
        keep_alive_period_in_seconds = 1000 # Keep instance warm to negate cold start for fast experimentation trials. Charge is incurred for warm instances
)

# train_input = TrainingInput(
#    training_dataset_s3_path,  content_type="text/csv", distribution = "ShardedByS3Key" if instance_count>1 else "FullyReplicated"
# )
# validation_input = TrainingInput(
#     validation_dataset_s3_path, content_type="text/csv"
# )
# xgb_script_mode_estimator.fit({"train": train_input,
#                                "validation": validation_input})

xgb_script_mode_estimator.fit(data_channels)

[01/22/25 19:04:13] INFO     Ignoring unnecessary Python version: py3.                            ]8;id=119301;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=854361;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#601\601]8;;\

                    INFO     Ignoring unnecessary instance type: ml.g5.2xlarge.                   ]8;id=430156;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=430431;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=188389;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=544972;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=973340;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=41383;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-01-22-19-04-13-860                                             

2025-01-22 19:04:18 Starting - Found matching resource for reuse..[2025-01-22 19:04:27.324 ip-172-31-85-198.ec2.internal:13 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-01-22 19:04:27.346 ip-172-31-85-198.ec2.internal:13 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-01-22:19:04:27:INFO] Imported framework sagemaker_xgboost_container.training
[2025-01-22:19:04:27:INFO] Invoking user training script.

2025-01-22 19:04:25 Downloading - Downloading the training image
2025-01-22 19:04:25 Training - Training image download completed. Training in progress.
2025-01-22 19:10:39 Uploading - Uploading generated training model
2025-01-22 19:10:39 Failed - Resource retained for reuse


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:38                                                                                   │
│                                                                                                  │
│   35 # xgb_script_mode_estimator.fit({"train": train_input,                                      │
│   36 #                                "validation": validation_input})                           │
│   37                                                                                             │
│ ❱ 38 xgb_script_mode_estimator.fit(data_channels)                                                │
│   39                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:166 in wrapper  │
│                                                                                                  │
│   163 │   │   │   │   │   caught_ex = e                                                          │
│   164 │   │   │   │   finally:                                                                   │
│   165 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 166 │   │   │   │   │   │   raise caught_ex                                                    │
│   167 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   168 │   │   │   else:                                                                          │
│   169 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:137 in wrapper  │
│                                                                                                  │
│   134 │   │   │   │   start_timer = perf_counter()                                               │
│   135 │   │   │   │   try:                                                                       │
│   136 │   │   │   │   │   # Call the original function                                           │
│ ❱ 137 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   138 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   139 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   140 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/estimator

In [55]:
# Specify FSx Lustre file system id.
file_system_id = "fs-0a234d2f11901041d"
# Specify FSx Lustre mount id.
fsx_mount_id = 'hjv3rb4v'

# Directory path for input data on the file system. 
file_system_directory_path_train = f'/{fsx_mount_id}/ucheegbe/train_reg'
file_system_directory_path_test = f'/{fsx_mount_id}/ucheegbe/test_reg'
print(f'FSx file-system data input path:{file_system_directory_path_train}')

# Specify the access mode of the mount of the directory associated with the file system. 
# Directory must be mounted 'ro'(read-only).
file_system_access_mode = 'ro'

# Specify your file system type.
file_system_type = 'FSxLustre'

from sagemaker.inputs import FileSystemInput
train = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path_train,
    file_system_access_mode=file_system_access_mode
)
val = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path_test,
    file_system_access_mode=file_system_access_mode
)

data_channels = {'train': train, "validation": val}
print(data_channels)

FSx file-system data input path:/hjv3rb4v/ucheegbe/train_reg
{'train': <sagemaker.inputs.FileSystemInput object at 0x7f40d12d7090>, 'validation': <sagemaker.inputs.FileSystemInput object at 0x7f40d12d7450>}


In [57]:
import boto3

client = boto3.client('fsx')
client.describe_file_systems(
    FileSystemIds=[
        'fs-0a234d2f11901041d',
    ],
  
)

# client.describe_data_repository_associations(
#     AssociationIds=[
#         'dra-08f6c4a32430c3a48',
#     ],   
# )

{'FileSystems': [{'OwnerId': '715253196401',
   'CreationTime': datetime.datetime(2024, 11, 18, 23, 42, 56, 237000, tzinfo=tzlocal()),
   'FileSystemId': 'fs-0a234d2f11901041d',
   'FileSystemType': 'LUSTRE',
   'Lifecycle': 'AVAILABLE',
   'StorageCapacity': 1200,
   'StorageType': 'SSD',
   'VpcId': 'vpc-09cfae9e123f5e3e4',
   'SubnetIds': ['subnet-01e40580335386c69'],
   'NetworkInterfaceIds': ['eni-01768b8172c41109e', 'eni-0834ad43b8263d99d'],
   'DNSName': 'fs-0a234d2f11901041d.fsx.us-east-1.amazonaws.com',
   'KmsKeyId': 'arn:aws:kms:us-east-1:715253196401:key/792b3b17-944b-4734-81f6-1db6889b188c',
   'ResourceARN': 'arn:aws:fsx:us-east-1:715253196401:file-system/fs-0a234d2f11901041d',
   'Tags': [{'Key': 'Name', 'Value': 'ucheegbe'}],
   'LustreConfiguration': {'WeeklyMaintenanceStartTime': '1:08:30',
    'DeploymentType': 'PERSISTENT_2',
    'PerUnitStorageThroughput': 125,
    'MountName': 'hjv3rb4v',
    'CopyTagsToBackups': False,
    'DataCompressionType': 'NONE',
    'LogC

In [59]:
import os
import boto3
import re
import sagemaker
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

role = sagemaker.get_execution_role()
region = sagemaker.Session().boto_region_name
session = Session()

# bucket = "<Specify S3 Bucket>"
# prefix = "<Specify S3 prefix>"
output_path = "s3://{}/{}/{}/output".format(bucket, prefix, "algo-dist-xgb")
hyperparams = {
    "objective": "binary:logistic",
    "num_round": "500",
    "verbosity": "3",
    "tree_method": "gpu_hist",
    "eval_metric": "auc",
    "use_dask_gpu_training": "true"
}


# output_path = "s3://{}/{}/output".format(bucket, prefix)

content_type = "text/csv"
instance_type = "ml.g5.2xlarge"

xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")
xgb_script_mode_estimator = sagemaker.estimator.Estimator(
    image_uri=xgboost_container,
    hyperparameters=hyperparams,
    role=role,
    instance_count=1,
    instance_type=instance_type,
    output_path=output_path,
    max_run=7200,
    # volume_size=200,
    keep_alive_period_in_seconds = 1000,
    subnets=['subnet-01e40580335386c69'],
    security_group_ids=['sg-04544eabba5214ad3']

)

train_input = TrainingInput(
    "s3://sagemaker-us-east-1-715253196401/sagemaker/DEMO-churn-dt/train2/", content_type=content_type
)

validation_input = TrainingInput(
    "s3://sagemaker-us-east-1-715253196401/sagemaker/DEMO-churn-dt/validation2/data.csv", content_type=content_type
)

# xgb_script_mode_estimator.fit({"train": train_input, "validation": validation_input})
xgb_script_mode_estimator.fit(data_channels)

[01/22/25 17:30:12] INFO     Ignoring unnecessary instance type: None.                            ]8;id=300791;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=394664;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=720732;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=792606;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=340194;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=669;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-01-22-17-30-12-322                                             

2025-01-22 17:30:13 Starting - Starting the training job
2025-01-22 17:30:13 Pending - Training job waiting for capacity......
2025-01-22 17:31:07 Pending - Preparing the instances for training...
2025-01-22 17:31:37 Downloading - Downloading input data......
2025-01-22 17:32:42 Training - Training image download completed. Training in progress..[2025-01-22 17:32:47.034 ip-172-31-87-90.ec2.internal:13 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-01-22 17:32:47.055 ip-172-31-87-90.ec2.internal:13 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-01-22:17:32:47:INFO] Imported framework sagemaker_xgboost_container.training
[2025-01-22:17:32:47:INFO] Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
[2025-01-22:17:32:47:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2025-01-22:17:32:47:INFO] Failed to parse hyperparameter tree_method value gpu_hist to Json.
R

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:56                                                                                   │
│                                                                                                  │
│   53 )                                                                                           │
│   54                                                                                             │
│   55 # xgb_script_mode_estimator.fit({"train": train_input, "validation": validation_input})     │
│ ❱ 56 xgb_script_mode_estimator.fit(data_channels)                                                │
│   57                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:166 in wrapper  │
│                                                                                                  │
│   163 │   │   │   │   │   caught_ex = e                                                          │
│   164 │   │   │   │   finally:                                                                   │
│   165 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 166 │   │   │   │   │   │   raise caught_ex                                                    │
│   167 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   168 │   │   │   else:                                                                          │
│   169 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py:137 in wrapper  │
│                                                                                                  │
│   134 │   │   │   │   start_timer = perf_counter()                                               │
│   135 │   │   │   │   try:                                                                       │
│   136 │   │   │   │   │   # Call the original function                                           │
│ ❱ 137 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   138 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   139 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   140 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/estimator

## Deploy the XGBoost model
Once the training is done, SageMaker packages your model artifacts along with any dependencies used for training including the `inference,py` which we will use as our inference logic. It is also possible to pass a seperate inference logic if you wish so.
After training we deploy the model using the Estimator object and point to our inference script for serving the model. Here we have defined SaegMaker specific functions `model_fn`, `predict_fn` to load and make prediction on the model.

In [68]:
predictor = xgb_script_mode_estimator.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.xlarge", 
    entry_point="inference.py", # path to inference script within the model package
)

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-10-28-03-37-11-315
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-10-28-03-37-11-315
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-10-28-03-37-11-315


------!

In [85]:
# Load test Data
import pandas as pd
features=pd.read_csv("test.csv").iloc[:,1:]
num_examples=len(features)

In [86]:
content_type = "text/csv"
import boto3
import json
import numpy as np

predictor.serializer = sagemaker.serializers.CSVSerializer()

def query_endpoint(encoded_tabular_data, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType=content_type,
        Body=encoded_tabular_data,
    )
    return response

# split the test data into smaller size of batches to query the endpoint if test data has large size.
batch_size = 1500
predict_prob = []
for i in np.arange(0, num_examples, step=batch_size):
    payload=features.iloc[i : (i + batch_size), :].to_csv(header=False, index=False).strip()
    predict_prob.extend(predictor.predict(payload))

In [ ]:
predict_prob

# Automatic model Tuning

Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. For example, suppose that you want to solve a binary classification problem on this marketing dataset. Your goal is to maximize the area under the curve (auc) metric of the algorithm by training an XGBoost Algorithm model. You don't know which values of the eta, alpha, min_child_weight, and max_depth hyperparameters to use to train the best model. To find the best values for these hyperparameters, you can specify ranges of values that Amazon SageMaker hyperparameter tuning searches to find the combination of values that results in the training job that performs the best as measured by the objective metric that you chose. Hyperparameter tuning launches training jobs that use hyperparameter values in the ranges that you specified, and returns the training job with highest auc.

In [87]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}

objective_metric_name = 'validation:auc'

In [89]:
tuner = HyperparameterTuner(xgb_script_mode_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=6, # maximum number of canditate jobs
                            max_parallel_jobs=3) # number of jobs to execute in parallel
tuner.fit({'train': train_input, 'validation': validation_input})

INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-241028-0353


.............................................................................!


In [ ]:
inference_instance_type = "ml.m5.xlarge"

# Use the estimator from the previous step to deploy to a SageMaker endpoint
predictor = tuner.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
)

In [91]:
# Load test Data
import pandas as pd
features=pd.read_csv("test.csv").iloc[:,1:]
num_examples=len(features)

In [92]:
content_type = "text/csv"
import boto3
import json
import numpy as np

predictor.serializer = sagemaker.serializers.CSVSerializer()


# split the test data into smaller size of batches to query the endpoint if test data has large size.
batch_size = 1500
predict_prob = []
for i in np.arange(0, num_examples, step=batch_size):
    payload=features.iloc[i : (i + batch_size), :].to_csv(header=False, index=False).strip()
    predict_prob.extend(predictor.predict(payload))

In [ ]:
predict_prob